In [1]:
import PySimpleGUI as sg
import vlc
from time import sleep
import os
from sys import platform

#----------- TEST MEDIA -----------#
import test_data
song1 = test_data.song1 # test a local path
song2 = test_data.song2 # test a remote path 

#----------- SETUP -----------#
# I don't need to reference window 1, I know it's at 0,0

number_of_media = 8
# locations = [(0,0), (0,720), (1280,0), (1280,720)]
locations = [(0,0), (0,720), (1280,0), (1280,720), (-2560,380), (-1280,380), (-2560,1000), (-1280,1000)]


#----------- GUI -----------#
def layout(number):
    # can't use the same layout each time, but it works if func just takes an unused param
    return [
        [sg.Graph(canvas_size=(1280,720), graph_bottom_left=(0,0), graph_top_right=(0,0), key='-G-')],
        [sg.Button('Exit', size=(10,1), pad=(0,0))]]

window_names = {}
instance_names = {}
player_names = {}
media_names = {}

for i in range(1, number_of_media + 1):
    window_names['window' + str(i)] = sg.Window(str(i), layout(i), finalize=True, location=locations[i-1])
    instance_names['instance' + str(i)] = vlc.Instance()
    player_names['player' + str(i)] = instance_names['instance' + str(i)].media_player_new()
    media_names['media' + str(i)] = instance_names['instance' + str(i)].media_new(song1)
    player_names['player' + str(i)].set_hwnd(window_names['window' + str(i)]['-G-'].Widget.winfo_id())
    player_names['player' + str(i)].set_media(media_names['media' + str(i)])


# #----------- MAIN LOOP -----------#

i, paused = 0, [False, False]

while True:
    window, event, values = sg.read_all_windows(timeout=1000)
    for i in range(1,number_of_media + 1):
        player_names['player' + str(i)].play()
    # Not sure this closes OK but might not be relevant for deployment
    if (window_names['window' + str(i)] == sg.WIN_CLOSED) or (event == 'Exit'):
        break

# #----------- CLEANUP -----------#
for i in range(1,number_of_media + 1):
    player_names['player' + str(i)].stop()
    player_names['player' + str(i)].release()
    instance_names['instance' + str(i)].release()
    window_names['window' + str(i)].close()
